In [2]:
from langchain.llms import OpenAI
import os

In [3]:
os.environ['OPENAI_API_KEY'] =" " # Add your key here

In [11]:
llm=OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6)

In [12]:
text ="what is the capital of France?"

In [13]:
print(llm.predict(text))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Hugging face

In [19]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "" # add your key here

In [22]:
from langchain.llms import HuggingFaceHub
llm_huggingface= HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0.6,"max_length":64})

In [23]:
output= llm_huggingface.predict("what is the capital of France?")

In [24]:
print(output)

paris


## Prompt templates and LLM chain

In [27]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables =["country"],template ="Tell me the capital of this country - {country}")
prompt_template.format(country="India")

'Tell me the capital of this country - India'

In [30]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm_huggingface,prompt=prompt_template)
chain.invoke("india")

{'country': 'india', 'text': 'chennai'}

## Combining multiple chain using simple sequential chain

In [54]:
capital_prompt  =PromptTemplate(input_variables=["country"],template="Please list the capital of the {country}")
capital_chain  = LLMChain(llm=llm_huggingface,prompt=capital_prompt,)
famous_food_template = PromptTemplate(input_variables=["capital"],template="Suggest me a must try food from this {capital}")

In [55]:
famous_food_chain = LLMChain(llm=llm_huggingface,prompt=famous_food_template,)

In [56]:
from langchain.chains import SimpleSequentialChain
sim_seq_chain = SimpleSequentialChain(chains=[capital_chain,famous_food_chain])
sim_seq_chain.invoke(
    "France"
)

{'input': 'France', 'output': 'i would recommend the risotto'}

## Sequential chain

In [58]:
capital_prompt  =PromptTemplate(input_variables=["country"],template="Please list the capital of the {country}")
capital_chain  = LLMChain(llm=llm_huggingface,prompt=capital_prompt,output_key="capital")
famous_food_template = PromptTemplate(input_variables=["capital"],template="Suggest me a must try food from this {capital}")
famous_food_chain = LLMChain(llm=llm_huggingface,prompt=famous_food_template,output_key="food")

In [62]:
from langchain.chains import SequentialChain
Seq_chain = SequentialChain(chains=[capital_chain,famous_food_chain],input_variables=["country"],output_variables=["capital","food"])

In [63]:
Seq_chain({"country":"France"})

{'country': 'France',
 'capital': 'paris',
 'food': 'i would recommend the risotto'}

# Chatmodels with ChatOpenAI

In [64]:
from langchain.chat_models import Chat